<a href="https://colab.research.google.com/github/OscoLP/DeepLearning-RemoteSensing/blob/main/DNN_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***Mount GoogleCollabs drive***

In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive')

***Read the .csv file***

In [ ]:
import pandas as pd

train = pd.read_csv('gdrive/My Drive/Colab Notebooks/Train.csv')
x_train = train.iloc[:, 0:-1].values
y_train = train.iloc[:, -1].values

test = pd.read_csv('gdrive/My Drive/Colab Notebooks/Test.csv')
x_test = test.iloc[:, 0:-1].values
y_test = test.iloc[:, -1].values

***Normalize the data***

In [ ]:
from sklearn.preprocessing import MinMaxScaler

sc = MinMaxScaler(feature_range = (0, 1))
x_train = sc.fit_transform(x_train)
x_test = sc.fit_transform(x_test)

***Import the model (keras)***

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

***Define the model structure***

In [ ]:
model = Sequential()

model.add(Dense(128, input_shape=(x_train.shape[1:]), activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(units = 2, activation='softmax'))

model.summary()

***Compile and train the model***

In [ ]:
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])

history = model.fit(x_train, y_train, epochs = 100, batch_size = 32, verbose = 1, shufle = 'TRUE', validation_split= 0.2)

***Visualize the training performance***

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.plot(history.history['loss'], label='Loss (training data)')
plt.plot(history.history['val_loss'], label='Loss (validation data)')
plt.title('Historic - Loss')
plt.ylabel('Loss value')
plt.xlabel('No. epoch')
plt.legend(loc="upper right")

plt.figure()
plt.plot(history.history['accuracy'], label='Accuracy (training data)')
plt.plot(history.history['val_accuracy'], label='Accuracy (validation data)')
plt.title('Historic - Acc')
plt.ylabel('Acc value')
plt.xlabel('No. epoch')
plt.legend(loc="best")

plt.show()

***Calculate the model accuracy***

In [ ]:
_, accuracy = model.evaluate(x_test, y_test)
print('Accuracy: %.2f' % (accuracy * 100))

***Print the model predictions***

In [ ]:
y_pred = model.predict(x_test)
print(y_pred)

***Print the model rounded predictions***

In [ ]:
import numpy as np
import itertools

y_rounded_predictions = np.argmax(y_pred, axis=-1)
y_initial_labels = test.iloc[:, -1].values

print(y_rounded_predictions)

***Visualize the confusion matrix***

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

y_rounded_predictions = np.argmax(y_pred, axis=-1)
y_initial_labels = test.iloc[:, -1].values

plt.figure()
cm = confusion_matrix(y_initial_labels, y_rounded_predictions)
def plot_confusion_matrix(cm, classes,
                        normalize=False,
                        title='Confusion Matrix',
                        cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 3.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
cm_plot_labels = ['0','1']

plot_confusion_matrix(cm=cm, classes=cm_plot_labels, title='Confusion Matrix')

plt.show()